# SentencePiece example

## Introduction

このnotebookは言語モデル（ここではT5を想定）に使用するTokenizerを作成するサンプルノートブックです。    
SagemakerNotebookインスタンスで動作検証していますがローカル含め他の環境でも動作すると思います。

以下を参考にしています

- https://github.com/google/sentencepiece
- https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install tensorflow-datasets==4.4.0

In [ ]:
!git clone https://github.com/google/sentencepiece

以下のコマンドをターミナルから実行してSentencePieceをBuildしてください

% cd sentencepiece    
% mkdir build    
% cd build    
% cmake ..    
% make -j $(nproc)    
% sudo make install    
% sudo ldconfig -v    

In [ ]:
import os

OUTPUT_DIR = './src/japanese-t5-base'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

ここでは'wikipedia/20190301.ja'を使用します。

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

ds = tfds.load(
    #name='wikipedia/20201201.ja', 
    name='wikipedia/20190301.ja', 
    shuffle_files=True,
    download=True,
    try_gcs=True
)

In [ ]:
train_ds = ds["train"].batch(128).prefetch(10)

In [ ]:
all_titles = []
all_texts = []


for example in tfds.as_numpy(train_ds):
    titles, texts = example["title"], example["text"]
    for title, text in zip(titles, texts):
        all_titles.append(title.decode('utf-8'))
        all_texts.append(text.decode('utf-8'))

        
with open("input.txt", "w") as f:
    for text in all_texts:
        lines = [line.strip() for line in text.split("\n")]
        for line in lines:
            if len(line) == 0:
                continue
            f.write(line + "\n")

In [ ]:
del ds, train_ds

入力データを一行一文形式へ加工

In [ ]:
%%bash
cat << EOF > preprocess.sh
#!/bin/bash
FILE=\$1
if [ \$# -ne 1 ]; then
  echo "Usage: ./preprocess.sh INPUT_TEXT"
  exit 1
fi
echo "Processing \${FILE}"
sed -i -e '/^$/d; /<doc id/,+1d; s/<\/doc>//g' \${FILE}
sed -i -e 's/ *$//g; s/。\([^」|)|）|"]\)/。\n\1/g; s/^[ ]*//g' \${FILE}
sed -i -e '/^。/d' \${FILE}
sed -i -e 's/\(.*\)/\L\1/' \${FILE}
EOF
chmod 744 preprocess.sh
./preprocess.sh input.txt

言語モデル（T5）が`build_sentencepiece_model.sh`のflagsを使って、ID=0がpadding、ID=1がEOS、ID=2がUNKになるように予約して構築されていることを前提にしているので、そのようにする

**この処理はメモリを消費します。エラーが出る際はマシンのメモリを大きくする or 以下をコマンドに追加してサンプリングしてください**

--input_sentence_size=12000000 
--shuffle_input_sentence=true

ちなみに--model_prefix=spmだとHuggingFaceでTokenizerをロードする際にエラーとなるので注意してください

In [ ]:
!/usr/local/bin/spm_train --input=./input.txt --model_prefix="spiece" --vocab_size=32000 --character_coverage=0.9995 --model_type=unigram --pad_id=0 --eos_id=1 --unk_id=2 --bos_id=-1 

In [ ]:
!mv spiece.model spiece.vocab ./src/japanese-t5-base